In [446]:
#pip install barnum


In [467]:
import pandas as pd
import pgeocode
import re
import random
import requests
import json
from random import randrange  
from barnum import gen_data

In [469]:
def get_geojson(): 
    features = []
    for i in range(0,500):
        zp, city, state  = gen_data.create_city_state_zip()
        z = nomi.query_postal_code(f'{zp}')
        nomi = pgeocode.Nominatim('us')
        qpc = nomi.query_postal_code(f'{zp}')    
        feature = {
            'type':'Feature',
            'geometry':{ "type": "Point",  "coordinates": [qpc['longitude'], qpc['latitude']]  },
            "properties": {  "name": city  }
            } 
        features.append(feature)
    df =pd.DataFrame(features) 
    js = df.to_json('data/locs.json', orient='records')
    df    

In [470]:
service_types = { 'SVOD':.50,'AVOD':.27, 'TVOD':.15, 'VOD':.09  }  
service_distribution = {
    'amazon':.08,'hulu':.12, 'netflix':.31, 'youtube':.21, 'other':.28    
}   
nps = {
    '18-34': {'1':.18,'2':.31,'3':.32,'4':.12,'5':.03,'0':.04},
    '35-49': {'1':.30,'2':.30,'3':.20,'4':.06,'5':.05,'0':.09},
    '50-64': {'1':.38,'2':.35,'3':.13,'4':.03,'5':.01,'0':.11},
    '65-120': {'1':.47,'2':.26,'3':.09,'4':.04,'5':.01,'0':.13}
}
freq_dist = {
    '18-34': { '1-DAILY':.49,'2-FEW_TIMES_WEEK':.34,'3-FEWER_THREE_PER_MONTH':.17 },
    '35-49': { '1-DAILY':.38,'2-FEW_TIMES_WEEK':.43,'3-FEWER_THREE_PER_MONTH':.19 },
    '50-64': { '1-DAILY':.26,'2-FEW_TIMES_WEEK':.43,'3-FEWER_THREE_PER_MONTH':.31 },
    '65-120':{ '1-DAILY':.21,'2-FEW_TIMES_WEEK':.40,'3-FEWER_THREE_PER_MONTH':.39 }
}
def get_freq_key(age): 
    if age<35:
        return get_metric(freq_dist['18-34'])
    elif age<50:
        return get_metric(freq_dist['35-49'])    
    elif age<65:
        return get_metric(freq_dist['50-64'])   
    else:
        return get_metric(freq_dist['65-120'])  

def get_nps_key(age): 
    if age<35:
        nps_key=get_metric(nps['18-34'])
    elif age<50:
        nps_key=get_metric(nps['35-49'])    
    elif age<65:
        nps_key=get_metric(nps['50-64'])   
    else:
        nps_key=get_metric(nps['65-120']) 
    return nps_key 

In [471]:
response = requests.get('https://couch-potato-1.herokuapp.com/api/services')
df = pd.DataFrame(response.json())  
df = df.loc[~df.Service_Name.isin(['YouTube Premium','Prime Video','Netflix','Hulu'])] 
#df.loc[df.Service_Type == 'TVOD']
df.loc[df.Service_Name.isin(['FOX','FX', 'NBC']), 'Popular']  = 10
df = df.loc[df.Popular  > 0 ]
df = df.append({'Service_ID':'0','Service_Name':'Rent or Buy', 'Service_Type':'VOD'}, ignore_index=True)
df_otherser=df  
df_otherser


,Service_ID,Service_Name,Service_Type,Service_Img,Service_URL,Popular
0,1,Crackle,AVOD,https://img.reelgood.com/service-logos/crackle...,Crackle.com,10.0
1,3,IMDb TV,AVOD,https://img.reelgood.com/service-logos/imdb_tv...,IMDbTV.com,10.0
2,4,Peacock,AVOD,https://img.reelgood.com/service-logos/peacock...,Peacock.com,10.0
3,6,PlutoTV,AVOD,https://img.reelgood.com/service-logos/plutotv...,PlutoTV.com,10.0
4,8,Tubi,AVOD,https://img.reelgood.com/service-logos/tubi_tv...,Tubi.com,10.0
5,12,Apple TV+,SVOD,https://img.reelgood.com/service-logos/apple_t...,AppleTV.com,10.0
6,15,CBS All Access,SVOD,https://img.reelgood.com/service-logos/cbs_all...,CBSAllAccess.com,10.0
7,20,Discovery+,SVOD,https://img.reelgood.com/service-logos/discove...,Discovery.com,10.0
8,21,Disney+,SVOD,https://img.reelgood.com/service-logos/disney_...,Disney.com,10.0
9,24,fuboTV,SVOD,https://img.reelgood.com/service-logos/fubo_tv...,fuboTV.com,10.0


In [472]:
def get_metric_kv(d): 
    while(True): 
        i = randrange(len(d))
        if random.random() < list(d.values())[i]:
            k = list(d.keys())[i]
            v = list(d.values())[i]
            return k, v

In [473]:
def get_metric(d):
    k, v = get_metric_kv(d) 
    return k

In [474]:
def get_service_metric(d): 
    sn = ''
    while(True): 
        i = randrange(len(d))
        if random.random() < list(d.values())[i]:
            sn = list(d.keys())[i]
            break
    if sn == 'other': 
        #return sn
        while(True): 
            stype, v = get_metric_kv(service_types)  
            if random.random() < v: 
                break  
        df_Service_Names = df_otherser.loc[df_otherser.Service_Type==stype, ['Service_Name', 'Popular']]    
        idx = randrange(len(df_Service_Names)) 
        sn = df_Service_Names.iloc[idx]['Service_Name'] 
    return sn

In [475]:
diclist = []
genders = ['Male','Female']
recs = 10000 + randrange(30,1100) 
for i in range(0,recs):
    ser_dict = {}
    age = randrange(18,80)    
    while(True):
        if len(ser_dict) >= int(get_nps_key(age)):
            break
        m = get_service_metric(service_distribution) 
        if m not in ser_dict:
            ser_dict[m] = 1
            
    freq = get_freq_key(age)
    gen = genders[randrange(len(genders))] 
    zp, city, state  = gen_data.create_city_state_zip()
    fname, lname = gen_data.create_name(gender=f'{gen}')
    nomi = pgeocode.Nominatim('us')
    qpc = nomi.query_postal_code(f'{zp}') 
    diclist.append({'row':str(i)
                    , 'age': age
                    , 'username': f'{fname.lower()}_{lname.lower()}'
                    , 'first_name': fname
                    , 'last_name': lname
                    , 'gender': f'{gen}'
                    , 'city': city
                    , 'state': state
                    , 'zip': zp
                    , 'lat': qpc['latitude']
                    , 'lng': qpc['longitude']
                    , 'freq': freq
                    , 'services': ser_dict}) 
df_services = pd.DataFrame(diclist) 
ser = pd.json_normalize(df_services['services'])  
df = pd.merge(df_services, ser, left_index=True, right_index=True)
df_all = df

In [476]:
df = df.loc[df.services!={}] 
df = df.drop(columns=['first_name','last_name'])
df.to_csv(f'data/NIESEN_VOD_VIEWING_HABITS-V4.CSV', index=False)
df = pd.read_csv('data/NIESEN_VOD_VIEWING_HABITS-V4.CSV')
df

,row,age,username,gender,city,state,zip,lat,lng,freq,...,Crackle,CBS All Access,IMDb TV,Disney+,HBO MAX,NBC,Rent or Buy,FOX,Peacock,Tubi
0,0,62,fabian_couture,Male,Incline Village,NV,89451,39.2564,-119.9521,1-DAILY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,60,winford_underwood,Male,Dennard,AR,72629,35.7252,-92.5575,2-FEW_TIMES_WEEK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,34,darell_akers,Male,Ailey,GA,30410,32.2143,-82.4762,1-DAILY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,73,don_macon,Male,Double Springs,AL,35553,34.1387,-87.3974,1-DAILY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46,glenda_corona,Female,Silver Spring,MD,20997,38.9907,-77.0261,2-FEW_TIMES_WEEK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9860,10808,76,ursula_vanhorn,Female,Fort Yates,ND,58538,46.0905,-100.6516,3-FEWER_THREE_PER_MONTH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9861,10809,18,herlinda_meyer,Female,North Charleston,SC,29418,32.8929,-80.0458,1-DAILY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9862,10810,28,rod_golding,Male,Albany,NY,12256,42.6526,-73.7562,1-DAILY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9863,10811,58,dannie_corcoran,Male,Donnellson,IA,52625,40.6612,-91.5745,3-FEWER_THREE_PER_MONTH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [437]:
df.describe()

,row,age,zip,netflix,amazon,Showtime,youtube,fuboTV,Crackle,Discovery+,...,FOX,PlutoTV,HBO MAX,Starz,Disney+,IMDb TV,FX,Tubi,NBC,Peacock
count,10865.000000,10865.000000,10865.000000,5202.0,1537.0,435.0,3698.0,423.0,222.0,474.0,...,118.0,248.0,419.0,429.0,451.0,213.0,117.0,245.0,132.0,206.0
mean,5952.470410,47.657064,49078.150299,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,3436.981627,17.938762,27701.657166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,18.000000,501.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,2976.000000,32.000000,25654.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,5953.000000,47.000000,47928.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,8921.000000,63.000000,72365.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,11902.000000,79.000000,99950.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
